<img src="../common/fun_long_logo-01.png">

# Tutorial on uncertainty propagation in ThermoFun.

Operations in ThermoFun are overloaded with all error propagation equations, meaning that any assigned error in values and parameters will be automatically propagated into the final calculated property without the need for the user to explicitly calculate it.

Miron et al., (2023). ThermoFun: A C++/Python library for computing standard thermodynamic properties of substances and reactions across wide ranges of temperatures and pressures. Journal of Open Source Software, 8(83), 4624

[![DOI](https://joss.theoj.org/papers/10.21105/joss.04624/status.svg)](https://doi.org/10.21105/joss.04624)

### Import ThermoFun module
ThermoFun delivers thermodynamic properties of substances and reactions at the temperature and pressure of interest.

In [ ]:
import thermofun as fun

### Initialize a ThermoFun database record - 
This database has uncertainties assingned to the G0, S0, and Cp0 values of Ca+2 and CO3-2 aqueous species.

In [ ]:
# initalize using a local file
aq17 = fun.Database('../databases/aq17-thermofune.json')

### ThermoFun engine class object
For calculating properties of substances or reactions, at T and P other than reference values, we need a ThermoFun engine class object. This is initialized using a database.

In [ ]:
engine = fun.ThermoEngine(aq17)

### Calculate properties of Ca+2 and Calcite dissolution reaction (Calcite = Ca+2 + CO3-2). T = 298.15 K P = 1e5 Pa. 
ThermoFun engine uses SI units, so the values are for 50 C and 1 bar

In [ ]:
subst_prop = engine.thermoPropertiesSubstance(298.15, 1e5, "Ca+2")

### Extract the value
The return type of the function is an object that contains the thermodynamic properties, val - values; err - error; ddt - derivative with T; ddp - derivative with P.

In [ ]:
subst_prop.gibbs_energy.val

In [ ]:
subst_prop.gibbs_energy.err

In [ ]:
subst_prop.gibbs_energy.ddp

In [ ]:
subst_prop.entropy.val

In [ ]:
subst_prop.entropy.err

In [ ]:
subst_prop.gibbs_energy.ddt

### Calcite dissolution reaction (Calcite = Ca+2 + CO3-2), logK. print(f'logK (Cal = Ca+2 + CO3-2) is {logK.val}')

In [ ]:
reaction_properties = engine.thermoPropertiesReaction(298.15, 0, "Calcite = Ca+2 + CO3-2")
logK = reaction_properties.log_equilibrium_constant
print(f'logK (Calcite = Ca+2 + CO3-2) is {logK.val}')

The error from the properties of reactants is propagated to the reaction properties.

In [ ]:
logK.err

### Extract the reaction entropy and Gibbs energy. Check the derivative of Gibbs energy with temperature. print(f' Entropy of reaction {Sr.val} is -dGr/dT = {-Gr.ddt}')
The derivatives are corectly calculated for most methods in ThermoFun, if there is a discrepancy then the explicit values should be taken. 

In [ ]:
Sr = reaction_properties.reaction_entropy
Gr = reaction_properties.reaction_gibbs_energy

print(f' Entropy of reaction {Sr.val} (explicit) is -dGr/dT = {-Gr.ddt} (implicit)')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate x-axis values
T_values = np.arange(0, 201, 5)

In [ ]:
# Example data

logKs = [engine.thermoPropertiesReaction(T+273.15, 0, "Calcite = Ca+2 + CO3-2").log_equilibrium_constant.val for T in T_values]
logKerrors = [engine.thermoPropertiesReaction(T+273.15, 0, "Calcite = Ca+2 + CO3-2").log_equilibrium_constant.err for T in T_values]

# Convert lists to NumPy arrays
logKs = np.array(logKs)
logKerrors = np.array(logKerrors)


In [ ]:
logKerrors

In [ ]:
from common.plotting import set_plot_dimensions

In [ ]:
# Calculate upper and lower bounds for the error band
lower_bound = logKs - logKerrors
upper_bound = logKs + logKerrors

set_plot_dimensions()

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(T_values, logKs, color='blue', label='Calcite')
plt.fill_between(T_values, lower_bound, upper_bound, color='blue', alpha=0.3, label='Error Band')
plt.xlabel('Temperature')
plt.ylabel('logK')
plt.title('Calcite = Ca+2 + CO3-2')
plt.legend()
plt.grid(True)
#plt.yscale('log')  # Setting y-axis to logarithmic scale
plt.show()